# Named Entity Recognition

In [ ]:
# !pip install transformers

from transformers import *
import torch

tokenizer = XLMRobertaTokenizerFast.from_pretrained('xlm-roberta-base')
model = XLMRobertaForTokenClassification.from_pretrained(
    'asahi417/tner-xlm-roberta-base-ontonotes5')

ner = pipeline('ner', model=model, tokenizer=tokenizer, grouped_entities=True)

In [ ]:
sequence = """<p>1 <strong>Mr Louis Ng Kok Kwang</strong> asked&nbsp;the Minister for Health 
whether couples who have pre-implantation genetically screened embryos stored overseas can have 
their embryos shipped to Singapore given current travel restrictions during the 
pandemic.</p><p><strong>	The Parliamentary Secretary to the Minister for Health 
(Ms Rahayu Mahzam) (for the Minister for Health)</strong>: Happy International Women's 
Day to all! During the pandemic, MOH received appeals from some couples to import their 
pre-implantation genetically screened embryos stored overseas.&nbsp;</p><p>In reviewing 
each appeal, the Ministry considered whether processes and standards employed by overseas 
assisted reproduction (AR) centres are aligned to Singapore’s regulatory requirements under 
the Licensing Terms and Conditions for AR Services (AR LTCs). The Ministry may on an 
exceptional basis allow importation of the embryos, subject to conditions. These conditions 
include: (a) declaration by the overseas AR centre that the relevant requirements under the AR 
LTCs, including the handling, processing and storage of the embryos, are adhered to; (b) that 
no other findings besides the presence or absence of chromosomal aberrations are reported, and 
(c) proper documentation of the screening test results that were provided to the patient and 
attending physician in our local AR centres.&nbsp;</p><p>Local AR centres which receive the 
tested embryos must also continue to ensure compliance with the AR LTCs.</p>"""


entity_list = ner(sequence)
for idx in entity_list:
  idx['label'] = idx.pop('entity_group')

doc= {"text": sequence, "ents" : entity_list}


In [ ]:
import spacy
from spacy import displacy

displacy.render(doc, style="ent", manual=True, jupyter=True)

# Sentiment Analysis

In [29]:
!pip install transformers
from transformers import BertTokenizer, BertForSequenceClassification, pipeline

RAW_SEQUENCES = [
# Expected negative sentiment
"""Sir, I am proud that the Malay/Muslim community here remains strong and \
united despite many tests and challenges. It just that lately, some members of the community \
and our asatizah have complained that when crucial issues are voiced out or problems are \
raised, there are no clear answers from the authorities responsible for managing our welfare, \
needs and well-being.""",

# Expected positive sentiment
"""The difficult adjustments made by the community in response to COVID-19 \
was possible because our religious leadership was decisive and united. MUIS played a central \
role in this by issuing religious guidance early and rallying asatizah and mosque leaders to \
guide the community to adapt to the changing environment. While MUIS was monitoring \
developments and decisions of religious authorities around the world, we could not simply \
copy what others had done, but rather had to find our own solutions. When difficult decisions \
were made, the community came together and supported these decisions. Everyone played a part \
to encourage and guide one another – lawyers, doctors, community leaders and ordinary citizens, \
all rolled up our sleeves.""",

# Unclear example, negative topic but neutral sentiment
"""One of the concerns I have relates to the guidance of high-risk Malay/Muslim \
youths who often come from families with many issues. They may have insufficient role models at \
home for a variety of reasons. It is not uncommon for such youths to suffer low confidence, poor \
performance in studies, or be involved in activities that may lead them down the road of \
juvenile delinquency. """,

# Unclear example, positive topic but possible negative sentiment
"""Earlier this year, MCCY had organised "Emerging Stronger Together" and \
"Ciptasama" sessions for the Malay/Muslim community to share their views and suggestions \
in building a community of success and a better Singapore for the future. I note from the \
"Emerging Stronger" website that only 6% of the participants in these conversations were \
Malay. And I hope that more in our community can participate"""
]

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained(
    'textattack/bert-base-uncased-SST-2')

nlp = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)
sentiment = {'LABEL_0': "Negative", "LABEL_1": "Positive"}

for seq in RAW_SEQUENCES:
  result = nlp(seq)[0]
  print("----\n", seq)
  print(f"label: {sentiment[result['label']]}, with score: {round(result['score'], 4)}\n")

----
 Sir, I am proud that the Malay/Muslim community here remains strong and united despite many tests and challenges. It just that lately, some members of the community and our asatizah have complained that when crucial issues are voiced out or problems are raised, there are no clear answers from the authorities responsible for managing our welfare, needs and well-being.
label: Negative, with score: 0.886

----
 The difficult adjustments made by the community in response to COVID-19 was possible because our religious leadership was decisive and united. MUIS played a central role in this by issuing religious guidance early and rallying asatizah and mosque leaders to guide the community to adapt to the changing environment. While MUIS was monitoring developments and decisions of religious authorities around the world, we could not simply copy what others had done, but rather had to find our own solutions. When difficult decisions were made, the community came together and supported the

# Summarisation

In [1]:
!pip install transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

RAW_SEQUENCE = """<p>1 <strong>Mr Louis Ng Kok Kwang</strong> asked&nbsp;the Minister for Health \
whether couples who have pre-implantation genetically screened embryos stored overseas can have \
their embryos shipped to Singapore given current travel restrictions during the \
pandemic.</p><p><strong>	The Parliamentary Secretary to the Minister for Health \
(Ms Rahayu Mahzam) (for the Minister for Health)</strong>: Happy International Women's \
Day to all! During the pandemic, MOH received appeals from some couples to import their \
pre-implantation genetically screened embryos stored overseas.&nbsp;</p><p>In reviewing \
each appeal, the Ministry considered whether processes and standards employed by overseas \
assisted reproduction (AR) centres are aligned to Singapore’s regulatory requirements under \
the Licensing Terms and Conditions for AR Services (AR LTCs). The Ministry may on an \
exceptional basis allow importation of the embryos, subject to conditions. These conditions \
include: (a) declaration by the overseas AR centre that the relevant requirements under the AR \
LTCs, including the handling, processing and storage of the embryos, are adhered to; (b) that \
no other findings besides the presence or absence of chromosomal aberrations are reported, and \
(c) proper documentation of the screening test results that were provided to the patient and \
attending physician in our local AR centres.&nbsp;</p><p>Local AR centres which receive the \
tested embryos must also continue to ensure compliance with the AR LTCs.</p>"""

tokenizer = AutoTokenizer.from_pretrained("sshleifer/distilbart-cnn-12-6")
model = AutoModelForSeq2SeqLM.from_pretrained("sshleifer/distilbart-cnn-12-6")

summarizer = pipeline("summarization", model=model, tokenizer=tokenizer)

print(summarizer(RAW_SEQUENCE, max_length=130, min_length=30, do_sample=False))

[{'summary_text': ' Mr Louis Ng Kok Kwang asked the Minister for Health whether couples who have pre-implantation genetically screened embryos stored overseas can have their embryos shipped to Singapore given current travel restrictions during the pandemic .'}]
